This notebook generates randomized dictionary objects representing artificial observation blocks as desribed in [these pages](https://keckobservatory.atlassian.net/wiki/spaces/DSI/pages/808321035/Change+controlled+documents). 

Data is stored on a local MongoDB collection.

Finally a few function calls are made to query observation blocks

In [1]:
import numpy as np
from controller_helper import *
import pymongo
import random
import string
from itertools import product
import pprint
import pdb
from functools import wraps
import urllib
from getpass import getpass
seed = 1984739
random.seed(seed)

INST_MAPPING = { 
                 'DEIMOS': {'DE', 'DF'},
                 'ESI': {'EI'},
                 'HIRES': {'HI'},
                 'KCWI': {'KB', 'KF'}, 
                 'LRIS': {'LB', 'LR'},
                 'MOSFIRE': {'MF'},
                 'OSIRIS': {'OI', 'OS'},
                 'NIRES': {'NR', 'NI', 'NS'},
                 'NIRC2': {'N2', 'NC'},
                }

pis = [
"Michael Bluth",
"Lindsay Bluth-Fünke",
"Gob Bluth",
"George Michael Bluth",
"Maeby Fünke",
"Buster Bluth",
"Tobias Fünke",
"George Bluth Sr.",
"Lucille Bluth",
]

observers = [
"Narrator",
"Oscar Bluth",
"Lucille Austero",
"Barry Zuckerkorn",
"Kitty Sanchez",
"Steve Holt",
"Lupe",
"Annyong Bluth",
"Carl Weathers",
"Maggie Lizer",
"Stefan Gentles",
"Marta Estrella",
"Cindi Lightballoon",
"John Beard",
"Ann Veal",
"Wayne Jarvis",
"Dr. Fishman",
"Stan Sitwell",
"Sally Sitwell",
"Mort Meyers",
"Starla",
"Tony Wonder",
"Gene Parmesan",
"Terry Veal",
"Rita Leeds",
"Larry Middleman",
"Bob Loblaw",
"Ron Howard",
"DeBrie Bardeaux",
"Rebel Alley",
"Herbert Love",
"Marky Bark",
"Argyle Austero",
"Paul 'P-Hound' Huan",
"Mark Cherry",
"Murphy Brown Fünke",
"Lottie Dottie Da",
"Dusty Radler"
]

comments = [
"Here’s some money. Go see a star war.",
"I don’t understand the question and I won’t respond to it.",
"I am one of the few honest people I have ever known.",
"I’m a scholar. I enjoy scholarly pursuits.",
"I’ve made a huge tiny mistake.",
"I hear the jury’s still out on science.",
]

status = [
    "undefined", 
    "completed", 
    "broken",
    "invalid",
    "progressing",
    "inqueue",
]

kcwi_science = ['KCWI_ifu_sci_dither', 'KCWI_ifu_sci_stare']

semesters = [str(x)+y for x, y in product(range(2000,2004), ['A', 'B'])]
letters = string.ascii_lowercase

# random generators 
randString = lambda x=4: ''.join(random.choice(letters) for i in range(x))
randFloat = lambda mag=10: mag * random.uniform(0,1)
randBool = lambda: bool(random.choice([0,1, None]))
randInt = lambda lr=0, ur=100: random.randint(lr, ur)
randArrStr = lambda x=1, y=1: [randString(x) for _ in range(random.randint(1, y)) ]
optionalRandString = lambda x=4: random.choice([None, randString(x)])
optionalRandArrString = lambda x, y=1: random.choice([None, randArrStr(x, y)])
sampleInst = lambda: random.choice(list(INST_MAPPING.keys()))
randPI = lambda: random.choice(pis)
randObserver = lambda: random.choice(observers)
randSemester = lambda: random.choice(semesters)
randPIList = lambda x=1: lambda x=1: list(np.random.choice(pis, size=random.randint(1, x), replace=False))
randObserverList = lambda x=1: list(np.random.choice(observers, size=random.randint(1, x), replace=False))
randComment = lambda: random.choice(comments)
optionalRandComment = lambda: random.choice([None, randComment()])
randStatus = lambda: random.choice(status)
rand_kcwi_science = lambda: random.choice(status)
z_fill_number = lambda x, zf=2: str(x).zfill(2)

In [2]:


        
raDeg = z_fill_number(randInt(0, 360))
arcMinutes = z_fill_number(randInt(0, 60))
arcSeconds = z_fill_number(randInt(0, 60))

decDeg = z_fill_number(randInt(0, 90))
elevation = random.choice(['+', '-'])

def generate_ra():
    raDeg = z_fill_number(randInt(0, 360))
    arcMinutes = z_fill_number(randInt(0, 60))
    arcSeconds = z_fill_number(randInt(0, 60))
    ra = " ".join([raDeg, arcMinutes, arcSeconds])
    return ra

def generate_dec():
    arcMinutes = z_fill_number(randInt(0, 60))
    arcSeconds = z_fill_number(randInt(0, 60))
    decDeg = z_fill_number(randInt(0, 90))
    elevation = random.choice(['+', '-'])
    dec = elevation+" ".join([decDeg, arcMinutes, arcSeconds])
    return dec

Functions below generate observation blocks dictionary objects, and their sub components.

In [3]:
def remove_none_values_in_dict(method):
    '''None values in dict returned by method are removed
    '''
    @wraps(method)
    def remove_none(*args, **kw):
        result = method(*args, **kw)
        return {key: val for key, val in result.items() if val is not None}
    return remove_none

def generate_semester(sem, nLen, maxLen=6):
    return {'_id': sem,
            'semester': sem,
            'obs_id': randObserverList(maxLen), 
            'comment': optionalRandComment()
           }

def generate_semesters(nSem, nLen=5, maxLen=6):
    return [ generate_semester(sem, nLen, maxLen) for sem in semesters[0:nSem] ]

def generate_mag(nLen=2):
    return {'band': randString(nLen), 'mag': randFloat(nLen), 'comment': optionalRandComment()}

def generate_mags(maxMags=2):
    return [ generate_mag() for _ in range( random.randint( 1, maxMags ) ) ]

@remove_none_values_in_dict
def generate_observation(nLen, maxArr):
    '''not used atm'''
    schema = {
        'instrument': sampleInst(),
        'exposure_sequences': randArrStr(nLen, maxArr),
        'associations': randArrStr(nLen, maxArr),
        'comment': optionalRandComment()
    }
    return schema

@remove_none_values_in_dict
def generate_signature(maxArr):
    schema = {
        'pi': randPI(),
        'semester': randSemester(),
        'program': random.randint(0, 10),
        'observers': randObserverList(maxArr),
        'group': random.randint(0,3),
        'comment': optionalRandComment()
    }
    return schema

def generate_dither():
    dmin, dmax = [random.randint(-15, 15), random.randint(-15, 15)].sort()
    schema = {
        'min': dmin,
        'max': dmax,
        'letter': random.choice(string.lower_case).upper(),
        'guide': 'Guided'
    }
    return schema

@remove_none_values_in_dict
def generate_kcwi_science(nLen, maxArr):
    name = rand_kcwi_science()
    cam = random.choice([ "BL","BM","BH1","BH2", "RL","RM","RH1","RH2"])
    camIsBlue = cam[0] is 'B'
    cwave = random.randint(3500, 6500) if camIsBlue else random.randint(6500, 10000)
    schema = {
        "name": name,
        "version": "0.1",
        "det1_exptime": random.randint(0,3600),
        "det1_nexp": random.randint(0,99),
        "det2_exptime": random.randint(0,3600),
        "det2_next": random.randint(0,99),
        "cfg_cam_grating": random.choice([ "BL","BM","BH1","BH2", "RL","RM","RH1","RH2" ]),
        "cfg_cam_cwave": random.randint(6500,10000),
        "cfg_slicer": random.choice(["Small", "Medium", "Large"])
    }
    if 'dither' in name:
        schema["SEQ.DITARRAY"] = generate_dither()
        schema["SEQ.NDITHER"] = random.randint(0,99)
    return schema


def generate_kcwi_acquisiton(nLen, maxArr):
    schema = {
        "name": "KCWI_ifu_acq_direct",
        "version": "0.1",
        "script": "KCWI_ifu_acq_direct",
        "guider_po": random.choice( ["REF","IFU"] ),
        "guider_gs_ra": random.uniform(0, 24) % 1000,
        "guider_gs_dec": random.uniform(-90, 90) % 1000,
        "guider_gs_mode": random.choice(["Automatic", "Operator", "User"])
    }
    return schema

def generate_science(nLen, maxArr, inst='KCWI'):
    if inst=='KCWI':
        schema = generate_kcwi_science(nLen, maxArr)
    else:
        schema = generate_kcwi_science(nLen, maxArr) # fill this in later
    return schema

def generate_acquisition(nLen, maxArr, inst='KCWI'):
    if inst=='KCWI':
        schema = generate_kcwi_acquisiton(nLen, maxArr)
    else:
        schema = {
            'instrument_setup': randString(),
            'acquisition_method': randString(),
            'guider_selection': optionalRandString(),
            'ao_modes': optionalRandArrString(nLen, maxArr),
            'offset_stars': optionalRandArrString(nLen, maxArr),
            'slitmasks': optionalRandArrString(nLen, maxArr),
            'position_angles': optionalRandArrString(nLen, maxArr),
            'comment': optionalRandComment()
        }
    return schema

@remove_none_values_in_dict
def generate_target():
    schema = {
        'name': randString(), 
        'ra': generate_ra(), 
        'dec': generate_dec(), 
        'equinox': randFloat(), 
        'frame': randString(), 
        'ra_offset': randFloat(), 
        'dec_offset': randFloat(),
        'pa': randInt(),
        'pm_ra': randFloat(), 
        'pm_dec': randFloat(), 
        'epoch': randFloat(), 
        'obstime': randFloat(), 
        'mag': generate_mags(), 
        'wrap': optionalRandString(), 
        'd_ra': randFloat(), 
        'd_dec': randFloat(), 
        'comment': optionalRandComment()
    }
    return schema

@remove_none_values_in_dict
def generate_observation_block(nLen, maxArr, inst='KCWI', _id=None):
    schema = {
        'signature': generate_signature(maxArr),
        'version': "0.1",
        'target': random.choice( [ None, generate_target() ] ),
        'acquisition': random.choice( [ None, generate_acquisition( nLen, maxArr, inst ) ] ),
        'science': random.choice( [None, generate_science( nLen, maxArr, inst ) ] ),
        'associations': randArrStr( nLen, maxArr ),
        'priority': randFloat(100),
        'status': randStatus(),
        'comment': optionalRandComment()
    }
    if _id:
        schema['_id'] = _id
    return schema

In [4]:
# run this a few times to check if data is generating.
assert generate_observation_block(3,3), 'ob did not generate successfully'

In [5]:
generate_observation_block(3,3)

{'signature': {'pi': 'George Michael Bluth',
  'semester': '2000A',
  'program': 10,
  'observers': ['Stefan Gentles', 'Cindi Lightballoon'],
  'group': 2},
 'version': '0.1',
 'acquisition': {'name': 'KCWI_ifu_acq_direct',
  'version': '0.1',
  'script': 'KCWI_ifu_acq_direct',
  'guider_po': 'IFU',
  'guider_gs_ra': 10.15882926902218,
  'guider_gs_dec': 951.5545237814936,
  'guider_gs_mode': 'Operator'},
 'science': {'name': 'inqueue',
  'version': '0.1',
  'det1_exptime': 530,
  'det1_nexp': 46,
  'det2_exptime': 72,
  'det2_next': 37,
  'cfg_cam_grating': 'BH1',
  'cfg_cam_cwave': 9391,
  'cfg_slicer': 'Large'},
 'associations': ['ddu', 'iah', 'ate'],
 'priority': 92.47225486286533,
 'status': 'completed',
 'comment': 'I don’t understand the question and I won’t respond to it.'}

Database name is 'papahana' collection name is 'ob_block'. Create collection assumes to be assessed through port 27017.

In [6]:
dbName = 'papahana'
collName = 'ob_block'
remote=True # run on remote server (n)
coll = create_collection(dbName, collName, port=27017, remote=False)

In [7]:
coll.drop()
coll.create_index([('signature.pi', pymongo.DESCENDING)])
coll.create_index([('signature.semester', pymongo.DESCENDING)])
coll.create_index([('signature.program', pymongo.DESCENDING)])

'signature.program_-1'

In [8]:
nLen = 5
maxArr = 5
nOb = 100
inst='KCWI'
for idx in range(nOb):
    doc = generate_observation_block(nLen, maxArr, inst, str(idx))
    result = coll.insert_one(doc)

In [9]:
result.inserted_id

'99'

The following queries can be called by the API, returning dictionaries as JSON objects. The first objects of each query are printed out. You are encouraged to alter the search queries yourself. 

In [10]:
sig = {
		"pi" : "Buster Bluth",
		"semester" : "2001B",
		"program" : 2,
		"observers" : [
			"Herbert Love",
			"Oscar Bluth",
			"Tony Wonder",
			"Mort Meyers"
		],
		"container" : 2
	}

container = 2

queries = (
    coll.find(),
    find_by_pi(sig["pi"], coll),
    find_by_observer(sig['observers'][0], coll),
    find_by_observer_group(sig['observers'][0], container, coll)
)

for cursor in queries:
    obExample = cursor.next()
    pprint.pprint(obExample['signature'], depth=2)

{'group': 0,
 'observers': ['Dusty Radler', 'Mark Cherry'],
 'pi': 'Lucille Bluth',
 'program': 8,
 'semester': '2001A'}
{'comment': 'I’ve made a huge tiny mistake.',
 'group': 2,
 'observers': ['Stan Sitwell', 'Stefan Gentles'],
 'pi': 'Buster Bluth',
 'program': 3,
 'semester': '2000B'}
{'comment': 'I hear the jury’s still out on science.',
 'group': 1,
 'observers': ['Herbert Love',
               'Oscar Bluth',
               'Stefan Gentles',
               'Murphy Brown Fünke',
               'Sally Sitwell'],
 'pi': 'George Michael Bluth',
 'program': 2,
 'semester': '2001B'}
{'comment': 'I don’t understand the question and I won’t respond to it.',
 'group': 2,
 'observers': ['Mort Meyers',
               'Larry Middleman',
               'Herbert Love',
               'Marky Bark',
               'Lupe'],
 'pi': 'Buster Bluth',
 'program': 5,
 'semester': '2002B'}


In [11]:
aggregates = (
get_distinct_semesters(sig['observers'][0], coll),  # /semIds/{semester} (get) 
get_semesters_by_pi(sig['pi'], coll),  # /semIds (get) # Retrieves all the programs (semesters) associated with a PI
get_ob_by_semester(sig['semester'], coll), #  /semIds/{semester} (get) retrieve the programs for the given semester
get_ob_by_semester_observer(sig['semester'], sig['observers'][0], coll)  # /semIds/{semId} (get)  Retrieves the specified program
)

for cursor in aggregates:
    obExample = cursor.next()
    pprint.pprint(obExample, depth=2)

{'_id': {'semester': '2001B'}}
{'_id': '19', 'signature': {'semester': '2000B'}}
{'_id': '1',
 'associations': ['xwigg'],
 'priority': 30.39958322213412,
 'signature': {'group': 3,
               'observers': [...],
               'pi': 'Michael Bluth',
               'program': 4,
               'semester': '2001B'},
 'status': 'inqueue',
 'target': {'comment': 'Here’s some money. Go see a star war.',
            'd_dec': 5.544516392746337,
            'd_ra': 3.150773038166893,
            'dec': '+35 42 21',
            'dec_offset': 6.232069077765852,
            'epoch': 4.105509807084279,
            'equinox': 6.007772226520177,
            'frame': 'obse',
            'mag': [...],
            'name': 'awny',
            'obstime': 2.592726461365581,
            'pa': 45,
            'pm_dec': 0.5915399145778211,
            'pm_ra': 5.4090214272112345,
            'ra': '253 21 43',
            'ra_offset': 9.310676563426895},
 'version': '0.1'}
{'_id': '10',
 'associations': 

In [46]:
ob = list(get_ob_by_id('17', coll))[0]
ob['_id'] = str(101)
delete_observation_block(str(101), coll)
insert_observation_block(ob, coll)
newValues = {"$set": { "priority": 100 } }
update_observation_block(str(101), newValues, coll)
ob = list(get_ob_by_id('101', coll))
print(ob)

[{'_id': '101', 'signature': {'pi': 'Lucille Bluth', 'semester': '2000B', 'program': 3, 'observers': ['Kitty Sanchez', 'Stefan Gentles'], 'container': 2, 'comment': 'I don’t understand the question and I won’t respond to it.'}, 'observations': [{'instrument': 'DEIMOS', 'exposure_sequences': ['jtjlh'], 'associations': ['vbwau', 'esitr', 'cyvwv'], 'comment': 'I am one of the few honest people I have ever known.'}, {'instrument': 'NIRES', 'exposure_sequences': ['mxsmr'], 'associations': ['qpujw', 'ihsld', 'lvqia', 'uwbtu'], 'comment': 'I hear the jury’s still out on science.'}, {'instrument': 'LRIS', 'exposure_sequences': ['fdldu', 'dhmzz', 'utlif', 'skszj', 'tlvno'], 'associations': ['ngetk', 'lyhbv', 'cbtwr', 'tjcvi']}, {'instrument': 'NIRES', 'exposure_sequences': ['dmkgi', 'ezfih', 'gmwvi', 'rugrp', 'apumf'], 'associations': ['kdjun', 'ryjua', 'veabs', 'zhszz', 'rxrgh'], 'comment': 'I don’t understand the question and I won’t respond to it.'}], 'associations': ['gddqy', 'srasf', 'rrav